In [1]:
import numpy as np
#import scipy.stats
#import seaborn as sns
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
#import scikit_posthocs as sp
#import matplotlib.pyplot as plt
#from matplotlib import gridspec
#from matplotlib.transforms import BlendedGenericTransform
#import scikit_posthocs as sp
#from urllib.request import urlopen
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

import os

import keras
import keras.layers as layers
from keras import backend as K
import numpy as np

Using TensorFlow backend.


TODO
* Complex numbers work, but not for image processing & convolutions! Make this into a 2 deep image
* Check alignment between center of window and targets
* Consider edge behavior in terms of data windows (zeroes?)

* Evaluate different representations of complex numbers in terms of learnability



In [2]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Noisy\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
fft = librosa.stft(wav)

In [3]:
# Constants and settings
WINDOW_SIZE = 10
FFT_BINS = 1025

In [84]:
def stack_fft(fft):
    return np.stack([fft.real, fft.imag], axis=-1)

def merge_fft_img(fft_img):
    return fft_img[:,:,0] + 1j * fft_img[:,:,1]

# Creating data from clip wave file
def clip_frames(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav) # organized as bins, frames
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE,2)) # image is depth 2 for real & imaginary
    for i in range(0, fft.shape[1] - WINDOW_SIZE):
        frames = np.append(frames,[stack_fft(fft[:,i:i+WINDOW_SIZE])], axis=0)
    return frames


def clip_targets(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav)
    targets = np.empty(shape=(0,FFT_BINS))
    for i in range(half_win, fft.shape[1] - half_win):
        targets = np.append(targets, [fft[:,i+half_win:i+half_win+1].flatten()], axis=0)
    return targets

In [86]:
ft = clip_frames(test_file)
tt = clip_targets(test_file)
print(ft.shape, tt.shape)

(110, 1025, 10, 2) (110, 1025)


In [89]:
# Iterate over clean & noisy folders to create frames and targets
def create_data(root):
    clean_dir = root + "\\Clean\\"
    noisy_dir = root + "\\Noisy\\"
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE,2))
    targets = np.empty(shape=(0,FFT_BINS))
    for file in os.listdir(clean_dir)[0:2]:
        filename = os.fsdecode(file)
        frames = np.concatenate((frames,clip_frames(noisy_dir + file)), axis=0)
        targets = np.concatenate((targets,clip_targets(clean_dir + file)), axis=0)
    return frames, targets


In [90]:
frames, targets = create_data("Assets\\DataShareArchive\\Test")

In [97]:
# Define network

model = keras.Sequential(
     [
        layers.Conv2D(16, kernel_size=2, activation='relu', padding='valid', 
                      input_shape=[FFT_BINS,WINDOW_SIZE,2]),
        layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        layers.Conv2D(16, kernel_size=2, activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        layers.Flatten(),
        layers.Dense(FFT_BINS, activation='relu'),
        layers.Dense(150, activation='relu'),
        layers.Dense(FFT_BINS, activation='linear')
     ]
    )

model.compile(optimizer='rmsprop', loss='mse')

In [98]:
model.fit(frames, targets, epochs=100, batch_size=50)

Epoch 1/100


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


172/172 [==============================] - 1s 6ms/step - loss: 3.2623
Epoch 2/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1751
Epoch 3/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1720
Epoch 4/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1713
Epoch 5/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1704
Epoch 6/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1713
Epoch 7/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1677
Epoch 8/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1646
Epoch 9/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1619
Epoch 10/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1592
Epoch 11/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1550
Epoch 12/100
172/172 [==============================] - 0s 2ms/step - loss: 3.1504
Epoch 13/100
172/172 [===

Epoch 100/100
172/172 [==============================] - 1s 3ms/step - loss: 1.2662


In [75]:
    a = np.array([[1+2j, 3+4j, 1.5+2.5j],[5+6j,7+8j, 5.5+6.5j]])
    t = np.empty(shape=(a.shape[0],a.shape[1],0))
    print(t,t.shape)
    print(a,a.shape)
    x = np.stack([a.real,a.imag], axis=-1)
    #x = np.append(t,[a.imag,a.real], axis=-1)
    print(x, x.shape)
    print(x[0,0,0], x[0,0,1])

[] (2, 3, 0)
[[1. +2.j  3. +4.j  1.5+2.5j]
 [5. +6.j  7. +8.j  5.5+6.5j]] (2, 3)
[[[1.  2. ]
  [3.  4. ]
  [1.5 2.5]]

 [[5.  6. ]
  [7.  8. ]
  [5.5 6.5]]] (2, 3, 2)
1.0 2.0


In [29]:
b

matrix([[2., 4.],
        [6., 8.],
        [1., 3.],
        [5., 7.]])